In [238]:
class Node(object):
    
    def __init__(self):
        self.children = []
        self.setmin = None
        self.setmax = None
        
    def is_zero(self):
        return False
    
    def is_one(self):
        return False
    
    def prtchk(self):
        pass
    
    def prtvar(self, eqc):
        return eqc
    
    def minval(self):
        return None
    
    def maxval(self):
        return None
    
    def eval(self, vars):
        raise Exception()

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)        
    
class VariableReferenceNode(Node):
    
    def __init__(self, name):
        super(VariableReferenceNode, self).__init__()
        self.name = name
        self.index = ord(name) - ord('a')
        
    def prt(self, indent_level):
        print(indent_level + self.name, end='')
                
    def minval(self):
        return 1
    
    def maxval(self):
        return 9
    
    def eval(self, vars):
        return vars[self.index]
    
    def prtvar(self, eqc):
        if eqc > 0:
            print(self.name)
        return eqc

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.name == other.name
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

class LiteralNode(Node):
    
    def __init__(self, value):
        super(LiteralNode, self).__init__()
        self.value = value
        
    def is_zero(self):
        return self.value == 0
    
    def is_one(self):
        return self.value == 1
    
    def minval(self):
        return self.value
    
    def maxval(self):
        return self.value
    
    def eval(self, vars):
        return self.value
        
    def prt(self, indent_level):
        print(indent_level + str(self.value), end='')

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.value == other.value
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
        
class BinaryOpNode(Node):
    
    def __init__(self, op, left, right):
        super(BinaryOpNode, self).__init__()
        self.op = op
        self.children.append(left)
        self.children.append(right)
        
    def prt(self, indent_level):
        print(indent_level + '(')
        print(indent_level + ' ' + self.op + ',')
        self.children[0].prt(indent_level + ' ')
        print(',')
        self.children[1].prt(indent_level + ' ')
        print()
        print(indent_level + ')', end='')

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.op == other.op and self.children[0] == other.children[0] and self.children[1] == other.children[1]
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

        
class AddNode(BinaryOpNode):
    
    def __init__(self, left, right):
        super(AddNode, self).__init__('+', left, right)
        
    def minval(self):
        left_min = self.children[0].minval()
        right_min = self.children[1].minval()
        new_min = left_min + right_min
        if self.setmin is not None and self.setmin < new_min:
            return self.setmin
        return new_min
        
    def maxval(self):
        left_max = self.children[0].maxval()
        right_max = self.children[0].maxval()
        return left_max + right_max
    
    def eval(self, vars):
        return self.children[0].eval(vars) + self.children[1].eval(vars)
                
class MulNode(BinaryOpNode):
    
    def __init__(self, left, right):
        super(MulNode, self).__init__('*', left, right)

    def minval(self):
        left_min = self.children[0].minval()
        right_min = self.children[1].minval()
        if left_min >= 0 and right_min >= 0:
            new_min = left_min * right_min
        else:
            print(f'Bailing on MulNode min with {left_min},{right_min}')
        if self.setmin is not None and self.setmin < new_min:
            return self.setmin
        return new_min
        
    def maxval(self):
        left_max = self.children[0].maxval()
        right_max = self.children[0].maxval()
        if left_max is not None and left_max >= 0 and right_max is not None and right_max >= 0:
            return left_max * right_max
        else:
            print(f'Bailing on MulNode max with {left_max},{right_max}')

    def prtchk(self):
        print(f'Mul node with {self.children[0].minval()},{self.children[0].maxval()},{self.children[1].minval()},{self.children[1].maxval()}')

    def eval(self, vars):
        return self.children[0].eval(vars) * self.children[1].eval(vars)

class DivNode(BinaryOpNode):
    
    def __init__(self, left, right):
        super(DivNode, self).__init__('/', left, right)

    def minval(self):
        left_min = self.children[0].minval()
        right_max = self.children[1].maxval()
        if left_min > 0 and right_max > 0:
            new_min = left_min // right_max
        else:
            print(f'Bailing on DivNode min with {left_min},{right_max}')   
        if self.setmin is not None and self.setmin < new_min:
            return self.setmin
        return new_min
        
    def maxval(self):
        left_max = self.children[0].maxval()
        right_min = self.children[1].minval()
        if left_max is not None and right_min is not None and left_max > 0 and right_min > 0:
            return left_max // right_min
        else:
            print(f'Bailing on DivNode max with {left_max},{right_min}')

    def prtchk(self):
        print(f'Div node with {self.children[0].minval()},{self.children[0].maxval()},{self.children[1].minval()},{self.children[1].maxval()}')

    def eval(self, vars):
        return self.children[0].eval(vars) // self.children[1].eval(vars)
        
class ModNode(BinaryOpNode):
    
    def __init__(self, left, right):
        super(ModNode, self).__init__('%', left, right)
        
    def minval(self):
        return 0
    
    def maxval(self):
        return self.children[1].maxval()

    def prtchk(self):
        print(f'Mod node with {self.children[0].minval()},{self.children[0].maxval()},{self.children[1].minval()},{self.children[1].maxval()}')

    def eval(self, vars):
        return self.children[0].eval(vars) % self.children[1].eval(vars)        
        
class EqNode(BinaryOpNode):
    
    def __init__(self, left, right):
        super(EqNode, self).__init__('==', left, right)
        
    def minval(self):
        return 0
    
    def maxval(self):
        return 1

    def prtchk(self):
        print(f'Eq node with {self.children[0].minval()},{self.children[0].maxval()},{self.children[1].minval()},{self.children[1].maxval()}')

    def prtvar(self, eqc):
        return eqc + 1

    def eval(self, vars):
        if self.children[0].eval(vars) == self.children[1].eval(vars):
            return 1
        else:
            return 0
    
def prtchk(node):
    node.prtchk()
    for child in node.children:
        prtchk(child)
        
def prtvar(node, eqc = 0):
    eqc = node.prtvar(eqc)
    for child in node.children:
        prtvar(child, eqc)
        
class Alu(object):
    
    def __init__(self):
        self.variable_counter = 'a'
        self.w = LiteralNode(0)
        self.x = LiteralNode(0)
        self.y = LiteralNode(0)
        self.z = LiteralNode(0)

    def inp(self, dst):
        variable = self.variable_counter
        self.variable_counter = chr(ord(self.variable_counter) + 1)
        setattr(self, dst, VariableReferenceNode(variable))
        
    def evaluate(self, reg_or_lit):
        if type(reg_or_lit) == int:
            return LiteralNode(reg_or_lit)
        return getattr(self, reg_or_lit)
        
    def bothlit(self, a, b):
        return hasattr(a, 'value') and hasattr(b, 'value')
        
    def isvar(self, a):
        return hasattr(a, 'name')
    
    def islit(self, a):
        return hasattr(a, 'value')
        
    def add(self, a, b):
        left = self.evaluate(a)
        right = self.evaluate(b)
        if left.is_zero():
            return setattr(self, a, right)
        if right.is_zero():
            return setattr(self, a, left)
        if self.bothlit(left, right):
            return setattr(self, a, LiteralNode(left.value + right.value))
        setattr(self, a, AddNode(left, right))
        
    def mul(self, a, b):
        left = self.evaluate(a)
        right = self.evaluate(b)
        if left.is_zero() or right.is_zero():
            return setattr(self, a, LiteralNode(0))
        if left.is_one():
            return setattr(self, a, right)
        if right.is_one():
            return setattr(self, a, left)
        if self.bothlit(left, right):
            return setattr(self, a, LiteralNode(left.value * right.value))
        setattr(self, a, MulNode(left, right))
    
    def mod(self, a, b):
        left = self.evaluate(a)
        right = self.evaluate(b)
        if left == right:
            print('Perfect equality in a mod')
            return setattr(self, a, LiteralNode(0))
        if right.is_one():
            return setattr(self, a, LiteralNode(0))
        if left.is_zero():
            return setattr(self, a, LiteralNode(0))
        if right.is_zero():
            raise Exception("Logic error")
        if self.bothlit(left, right):
            return setattr(self, a, LiteralNode(left.value % right.value))
        left_max = left.minval()
        right_min = right.minval()
        if left_max is not None and right_min is not None and left_max < right_min:
            return setattr(self, a, left)
        setattr(self, a, ModNode(left, right))
    
    def div(self, a, b):
        left = self.evaluate(a)
        right = self.evaluate(b)
        if left == right:
            print('Perfect equality in a div')
            return setattr(self, a, LiteralNode(1))
        if right.is_one():
            return setattr(self, a, left)
        if right.is_zero():
            raise Exception("Logic error")
        if self.bothlit(left, right):
            return setattr(self, a, LiteralNode(left.value // right.value))
        if self.isvar(left) and self.islit(right) and (right.value > 10):
            return setattr(self, a, LiteralNode(0))
        left_max = left.maxval()
        right_min = right.minval()
        if left_max is not None and right_min is not None and left_max < right_min:
            return setattr(self, a, LiteralNode(0))
        setattr(self, a, DivNode(left, right))
    
    def eql(self, a, b):
        left = self.evaluate(a)
        right = self.evaluate(b)
        if self.isvar(left) and self.islit(right) and (right.value >= 10 or right.value < 0):
            return setattr(self, a, LiteralNode(0))
        if self.isvar(right) and self.islit(left) and (left.value >= 10 or left.value < 0):
            return setattr(self, a, LiteralNode(0))
        if self.bothlit(left, right):
            if left.value == right.value:
                return setattr(self, a, LiteralNode(1))
            else:
                return setattr(self, a, LiteralNode(0))
            
        if left == right:
            print('Perfect eq')
            return setattr(self, a, LiteralNode(1))
            
        left_min = left.minval()
        right_min = right.minval()
        left_max = left.maxval()
        right_max = right.maxval()
        if left_min is not None and left_max is not None and right_min is not None and right_max is not None:
            if (left_min > right_max) or (right_min > left_max):
                return setattr(self, a, LiteralNode(0))
            
        setattr(self, a, EqNode(left, right))

In [239]:
alu = Alu()

def inp(val):
    return alu.inp(val)

def mul(a, b):
    return alu.mul(a, b)

def mod(a, b):
    return alu.mod(a, b)

def div(a, b):
    return alu.div(a, b)

def add(a, b):
    return alu.add(a, b)

def eql(a, b):
    return alu.eql(a, b)

inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',13)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',13)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',11)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',10)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',15)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',5)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',-11)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',14)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',14)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',5)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',0)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',15)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',12)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',4)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',12)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',11)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',1)
add('x',14)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',1)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',-6)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',15)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',-10)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',12)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',-12)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',8)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',-3)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',14)
mul('y','x')
add('z','y')
inp('w')
mul('x',0)
add('x','z')
mod('x',26)
div('z',26)
add('x',-5)
eql('x','w')
eql('x',0)
mul('y',0)
add('y',25)
mul('y','x')
add('y',1)
mul('z','y')
mul('y',0)
add('y','w')
add('y',9)
mul('y','x')
add('z','y')


In [264]:
import itertools
# alu.w.prt('')
# print()
# alu.x.prt('')
# print()
# alu.y.prt('')
# print()
# alu.z.prt('')
# print()
# prtvar(alu.z)

# HERE IS WHERE I ABANDON ALL THAT PRETTY LOGIC ABOVE AND
# RESORT TO BRUTE FORCE :'(

largest = [1, 2, 9, 3, 4, 9, 9, 8, 9, 4, 9, 1, 9, 9]
smallest = [0, 0, 6, 0, 0, 5, 9, 0, 5, 0, 1, 1, 7, 8]
prefix = [1, 1, 7, 1, 1, 6, 9, 1, 6, 1, 2, 1, 8, 9]
print(''.join([str(s) for s in prefix]))
tester = prefix.copy()
digits_to_replace = [7, 8, 9, 10]
digits = [1, 2, 3, 4, 5, 6, 7, 8, 9]
digits_to_test = len(digits_to_replace)
perms = itertools.product(digits, repeat=digits_to_test)
minval = 100000
for perm in perms:
    for i, v in enumerate(perm):
        tester[digits_to_replace[i]] = v
    resul = alu.z.eval(tester)
    if resul == 0:
        print(str(resul) + ' ' + str(perm))
        break


11711691612189
0 (1, 6, 1, 2)
